Перед началом выполнения задания, нужно поставить все зависимости, для этого нужно установить requirements.
pip3 install -r requirements.txt

# Перед выполнением нужно установить "adb" с помощью коменды "apt install adb", и разрешить дебаг через usb на телефоне. Для этого нужно:
1. Перевести телефон в режим разработчика( Настройки > О телефоне > Версия ПО > Дополнительно, далее нажать 7 раз на номер сборки.)
2. в Меню разработчика разрешить отладку по usb

Баллы:
dynamic квантование - 10 + 10
Замена HardSwish - 15
Замена HardSigmoid - 15

In [2]:
import torch
import tensorflow as tf
from pathlib import Path
from torchvision.models.segmentation import DeepLabV3_MobileNet_V3_Large_Weights
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large

2024-05-13 11:19:40.440714: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 11:19:40.466980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 11:19:40.467001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 11:19:40.467720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 11:19:40.472200: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# качаем модельку 
model = deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT)
model.eval();

In [4]:
# сохраняем модельку в onnx
input_tensor = torch.rand((1, 3, 300, 300))
with torch.no_grad():
    output_fixed = model(input_tensor)['out']

torch.onnx.export(model, input_tensor, f=f"dv3_mnv3.onnx", export_params=True, input_names=['input'], 
                  do_constant_folding=True, opset_version=13, output_names=['output'])

In [9]:
# экспортим модельку в tf
!onnx2tf -i dv3_mnv3.onnx -o dv3_mnv3 -osd


Model optimizing started ============================================================
Installing onnxruntime by `/usr/bin/python3 -m pip install onnxruntime`, please 
wait for a moment..
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                   ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add               │ 10             │ 10               │
│ Concat            │ 4              │ 1                │
│ Constant          │ 156            │ 146              │
│ Conv              │ 72             │ 72               │
│ GlobalAveragePool │ 9              │ 

In [10]:
# сохраняем модель в tflite в fp32
converter = tf.lite.TFLiteConverter.from_saved_model("dv3_mnv3")
tflite_model_quant = converter.convert()
Path("dv3_mnv3_fp32.tflite").write_bytes(tflite_model_quant)

2024-05-12 21:37:24.791333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-12 21:37:24.794679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-12 21:37:24.794855: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

44115456

In [11]:
# сохраняем модель в tflite в fp16
converter = tf.lite.TFLiteConverter.from_saved_model("dv3_mnv3")
converter.target_spec.supported_types = [tf.float16]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()
Path("dv3_mnv3_fp16.tflite").write_bytes(tflite_model_quant)

2024-05-12 21:38:01.111959: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-12 21:38:01.111980: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-12 21:38:01.112100: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: dv3_mnv3
2024-05-12 21:38:01.122680: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-12 21:38:01.122699: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: dv3_mnv3
2024-05-12 21:38:01.141384: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-05-12 21:38:01.189190: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: dv3_mnv3
2024-05-12 21:38:01.216527: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 104427 microseconds.
Summary on

22103636

In [3]:
# Тут вам нужно сделать dynamic квантование. сохранить модельку с именем "dv3_mnv3_int8_dynamic.tflite" (10 баллов)
import glob 
import cv2
import numpy as np 

mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def representative_dataset():
    paths = glob.glob("/home/gvasserm/data/coco2017/train2017/*.jpg")[:200]
    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue
        img = cv2.resize(img, (300, 300))
        img = img[:, :, ::-1]  # Convert BGR to RGB
        img = np.expand_dims(img, axis=0).astype(np.float32)
        img = (img / 255.0 - mean) / std
        yield [img]  # Ensure this is a list of numpy arrays

converter = tf.lite.TFLiteConverter.from_saved_model("dv3_mnv3")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_model_quant = converter.convert()
Path("dv3_mnv3_int8_dynamic.tflite").write_bytes(tflite_model_quant)

2024-05-13 09:55:17.805215: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-13 09:55:17.805230: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: gvasserm-desk
2024-05-13 09:55:17.805232: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: gvasserm-desk
2024-05-13 09:55:17.805288: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.29.6
2024-05-13 09:55:17.805296: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 545.29.6
2024-05-13 09:55:17.805297: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 545.29.6
2024-05-13 09:55:18.361717: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-13 09:55:18.3617

11596120

In [ ]:
# Начнём замеры!
# Для начала проверим есть ли подключённые устройства
!adb devices

In [ ]:
# Скачаем бенчмарк
!wget https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/android_aarch64_benchmark_model

In [ ]:
# закинем на устройство модельи и бенчмарк
!chmod +x android_aarch64_benchmark_model
!adb push android_aarch64_benchmark_model /data/local/tmp/
!adb push dv3_mnv3_fp32.tflite /data/local/tmp/
!adb push dv3_mnv3_fp16.tflite /data/local/tmp/
!adb push dv3_mnv3_int8_dynamic.tflite /data/local/tmp/

In [ ]:
# запустим модельки в одном потоке на цпу, нужно дождаться вывода времени пример строки:
# INFO: Inference timings in us: Init: 55393, First inference: 131849, Warmup (avg): 120734, Inference (avg): 119747
!adb shell "cd /data/local/tmp \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp32.tflite  --num_threads=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp16.tflite --num_threads=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_int8_dynamic.tflite  --num_threads=1"

In [ ]:
# на NNAPI
!adb shell "cd /data/local/tmp \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp32.tflite  --num_threads=1 --use_nnapi=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp16.tflite --num_threads=1  --use_nnapi=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_int8_dynamic.tflite  --num_threads=1  --use_nnapi=1"

In [ ]:
# и на гпу
!adb shell "cd /data/local/tmp \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp32.tflite  --num_threads=1 --use_gpu=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fp16.tflite --num_threads=1  --use_gpu=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_int8_dynamic.tflite  --num_threads=1  --use_gpu=1"

In [ ]:
# В логах мы можем увидеть, что не все операиции операции запустились на GPU и большая часть запустилась 
# на xnnpack(да и там не все операции запустились),
# Время GPU не сильно отличается от CPU. По логам можно понять, что проблема кроется в RELU_0_TO_1. Так же если мы откроем 
# onnx модельку через netron то мы увидим в ней, активации HardSigmoid и HardSwish. хоть они и поддерживаются в tflite 
# они не поддерживаются в gpu делегате. https://www.tensorflow.org/lite/performance/gpu
# но благо это не такие сложные операции, и от них можно избавится с помощью relu6, умножения и сложенияы, это вам и предстоит сделать.

In [ ]:
# здесь вам предстоит написать код который заменяет операции HardSigmoid (15 баллов) и HardSwish (15 баллов) на эквивалентные.
# вы можете сделать это как в torch так и в onnx

In [9]:
import torch
import torch.nn as nn

class ReLU6_Sigmoid(nn.ReLU6):

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return nn.ReLU6().forward(input + 3.)* 0.16666667
    
class ReLU6_Swish(ReLU6_Sigmoid):

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return input * ReLU6_Sigmoid().forward(input)


def convert_hardswish_to_relu6(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.Hardswish):
            setattr(model, child_name, ReLU6_Swish())
        else:
            convert_hardswish_to_relu6(child)

def convert_hardsigmoid_to_relu6(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.Hardsigmoid):
            setattr(model, child_name, ReLU6_Sigmoid())
        else:
            convert_hardsigmoid_to_relu6(child)

model_fixed = deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT)
convert_hardswish_to_relu6(model_fixed)
convert_hardsigmoid_to_relu6(model_fixed)
model_fixed.eval()


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU6_Swish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stri

In [10]:
#сохраняем и конвертируем модельку
input_tensor = torch.rand((1, 3, 300, 300))
with torch.no_grad():
    output_fixed = model_fixed(input_tensor)['out']

torch.onnx.export(model_fixed, input_tensor, f=f"dv3_mnv3_fixed.onnx", export_params=True, input_names=['input'], 
                  do_constant_folding=True, opset_version=13, output_names=['output'])

In [11]:
!onnx2tf -i dv3_mnv3_fixed.onnx -o dv3_mnv3_fixed -osd


Model optimizing started ============================================================
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                   ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add               │ 38             │ 38               │
│ Clip              │ 28             │ 28               │
│ Concat            │ 4              │ 1                │
│ Constant          │ 268            │ 150              │
│ Conv              │ 72             │ 72               │
│ GlobalAveragePool │ 9              │ 9                │
│ Mul               │ 56             │ 56               │
│ Relu              │ 27             │ 27               │
│ Resize            │ 3              │ 3                │
│ Shape             │ 3              │ 0                │
│ Slice             │ 3              │ 0                │
│ Model Size        │ 42.1MiB        │ 42.1MiB        

In [12]:
FIXED_SAVED_MODEL_DIR = "dv3_mnv3_fixed"

In [13]:
# сохраняем модель в tflite в fp32
converter = tf.lite.TFLiteConverter.from_saved_model(FIXED_SAVED_MODEL_DIR)
tflite_model_quant = converter.convert()
Path("dv3_mnv3_fixed_fp32.tflite").write_bytes(tflite_model_quant)

# сохраняем модель в tflite в fp16
converter = tf.lite.TFLiteConverter.from_saved_model(FIXED_SAVED_MODEL_DIR)
converter.target_spec.supported_types = [tf.float16]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()
Path("dv3_mnv3_fixed_fp16.tflite").write_bytes(tflite_model_quant)



2024-05-13 11:31:54.906294: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-13 11:31:54.906311: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: gvasserm-desk
2024-05-13 11:31:54.906314: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: gvasserm-desk
2024-05-13 11:31:54.906341: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.29.6
2024-05-13 11:31:54.906348: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 545.29.6
2024-05-13 11:31:54.906350: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 545.29.6
2024-05-13 11:31:55.437287: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-13 11:31:55.4373

22100084

In [15]:
# здесь нужно сделать dynamic квантование пофикшеной модели (10 баллов)
import glob 
import cv2
import numpy as np 

mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def representative_dataset():
    paths = glob.glob("/home/gvasserm/data/coco2017/train2017/*.jpg")[:200]
    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue
        img = cv2.resize(img, (300, 300))
        img = img[:, :, ::-1]  # Convert BGR to RGB
        img = np.expand_dims(img, axis=0).astype(np.float32)
        img = (img / 255.0 - mean) / std
        yield [img]  # Ensure this is a list of numpy arrays

converter = tf.lite.TFLiteConverter.from_saved_model(FIXED_SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_model_quant = converter.convert()
Path("dv3_mnv3_fixed_int8.tflite").write_bytes(tflite_model_quant)

2024-05-13 11:34:18.840101: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-13 11:34:18.840121: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-13 11:34:18.840242: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: dv3_mnv3_fixed
2024-05-13 11:34:18.851023: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-13 11:34:18.851039: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: dv3_mnv3_fixed
2024-05-13 11:34:18.875835: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-05-13 11:34:18.925472: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: dv3_mnv3_fixed
2024-05-13 11:34:18.954232: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 113989 micros

11591768

In [ ]:
!adb push dv3_mnv3_fixed_fp32.tflite /data/local/tmp/
!adb push dv3_mnv3_fixed_fp16.tflite /data/local/tmp/
!adb push dv3_mnv3_fixed_int8_dynamic.tflite /data/local/tmp/

In [ ]:
!adb shell "cd /data/local/tmp \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_fp32.tflite  --num_threads=1 --use_gpu=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_fp16.tflite --num_threads=1  --use_gpu=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_int8_dynamic.tflite  --num_threads=1  --use_gpu=1"

In [ ]:
# можно увидеть что все ноды сконвертились и моделька стала работать куда быстрее

In [ ]:
!adb shell "cd /data/local/tmp \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_fp32.tflite  --num_threads=1 --use_nnapi=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_fp16.tflite --num_threads=1  --use_nnapi=1 \
    && echo '-------------------------------------------------' \
    && ./android_aarch64_benchmark_model --graph=dv3_mnv3_fixed_int8_dynamic.tflite  --num_threads=1  --use_nnapi=1"

In [ ]:
#Тоже самое можно увидеть и на других делегатах